In [1]:
#Importing all the required libraries

import nltk
from nltk import grammar, parse
import nltk.tag
from pprint import pprint
from collections import Counter



In [19]:
#Take the CONLL-Train data and extract from each sentence all chunks and the sequence of tags that make up that chunk.
#Function to read the file and tokenize the words to form chunks and sentences
def read_conll(filepath):
    result = []
    result1 = []
    result2 = []
    file = open(filepath)
    sentence = []
    sentence1 = ""
    sentence2 = []
    for line in file:
        line = line.strip('\n')
        if not line.strip(' '):
            result.append(sentence)
            result1.append(sentence1)
            result2.append(sentence2)
            sentence = []
            
            continue
        (word,pos,tag) = line.split(' ')
        sentence.append((pos,tag)) # appending only pos and tag
        sentence2.append((word,pos,tag)) #appending word,pos and tag
        sentence1= sentence1 + " " + word # taking only the words to form sentences
    return result,result1,result2
    

conll_train, words_train, x = read_conll('train.txt')
conll_test, words_test, tree_test = read_conll('test.txt')



In [3]:
#Function to form chunks and trees from the sentence and counting

def extract_chunks(words):
    tree = {}
    tokenized_sent = nltk.tokenize.word_tokenize(words)
    pos_tagged_sent = nltk.pos_tag(tokenized_sent)
    pos_tags = [t for (w,t) in pos_tagged_sent]
    ct = nltk.tag.CRFTagger()
    ct.train(conll_train,'crf.model')
    ct.evaluate(conll_test)
    chunks = ct.tag(pos_tags)
    counts = Counter( tag for word,  tag in chunks) #Counting the occurence of each tag in each sentence
    print(counts)
    chunked_sent = [(w,p,t) for (w,(p,t)) in zip(tokenized_sent,chunks)]
    #pprint(chunked_sent)
    tree = nltk.chunk.conlltags2tree(chunked_sent)
    #print(tree)
    return tree, chunked_sent

In [4]:
len(words_train)

8936

In [50]:
#Since the dataset is huge I am taking only 2 sentences here as it was taking too long to run
for i in range (2):
    tree, chunked_sent = extract_chunks(words_train[i])
    print(tree) #Printing the tree structure and the counter
   

/var/folders/9z/75yp7_695hg75lbf0tpmjs9w0000gn/T/ipykernel_4437/3309341858.py:8: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  ct.evaluate(conll_test)


Counter({'I-NP': 11, 'B-NP': 9, 'I-VP': 6, 'B-PP': 5, 'O': 3, 'B-VP': 2, 'B-ADJP': 1})
(S
  (NP Confidence/NN)
  (PP in/IN)
  (NP the/DT pound/NN)
  (VP is/VBZ widely/RB expected/VBN to/TO take/VB)
  (NP another/DT sharp/JJ dive/NN)
  (PP if/IN)
  (NP trade/NN figures/NNS)
  (PP for/IN)
  (NP September/NNP)
  ,/,
  (ADJP due/JJ)
  (PP for/IN)
  (NP release/NN tomorrow/NN)
  ,/,
  (VP fail/VBP to/TO show/VB)
  (NP a/DT substantial/JJ improvement/NN)
  (PP from/IN)
  (NP July/NNP and/CC August/NNP)
  (NP 's/POS near-record/JJ deficits/NNS)
  ./.)
Counter({'I-NP': 22, 'B-NP': 16, 'B-PP': 9, 'I-VP': 9, 'O': 4, 'B-VP': 3, 'B-ADJP': 1})
(S
  (NP Confidence/NN)
  (PP in/IN)
  (NP the/DT pound/NN)
  (VP is/VBZ widely/RB expected/VBN to/TO take/VB)
  (NP another/DT sharp/JJ dive/NN)
  (PP if/IN)
  (NP trade/NN figures/NNS)
  (PP for/IN)
  (NP September/NNP)
  ,/,
  (ADJP due/JJ)
  (PP for/IN)
  (NP release/NN tomorrow/NN)
  ,/,
  (VP fail/VBP to/TO show/VB)
  (NP a/DT substantial/JJ improvement

In [33]:
test_tree = nltk.chunk.conlltags2tree(tree_test[0])



In [49]:
#Writing grammar by hand from the above output
grammar = r"""NP: {<[CDJNP].*>+}
 {<DT|PRP.*>?(<JJ>)*<NN.*><PP>?}
 {<PRP>}
 {<DT>?<JJ>*<NN>}
 {<DT><NN>}
 VP:{<VBZ>*<VBN>*<TO>*<VB>}
 PP: {<IN><NP>}"""


In [48]:
#Trying to evaluate the grammar on the test tree
cp = nltk.RegexpParser(grammar)
cp.evaluate(test_tree)

/var/folders/9z/75yp7_695hg75lbf0tpmjs9w0000gn/T/ipykernel_4437/2876804960.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  cp.evaluate(test_tree)


AttributeError: 'tuple' object has no attribute 'leaves'

Error is happening due to some structure issue of the test_tree which I was stuck in.